## Install libraries of langchain, openai, pinecone-client.

In [1]:
!pip install -qU \
    langchain==0.1.6 langchain-community==0.0.19 langchain-core==0.1.23 \
    openai==1.6.1 \
    pinecone-client==3.1.0 \
    tiktoken==0.5.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-text-splitters 0.3.0 requires langchain-core<0.4.0,>=0.3.0, but you have langchain-core 0.1.23 which is 

In [2]:
import os
import pandas as pd
import time
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.embeddings.openai import OpenAIEmbeddings
from tqdm.auto import tqdm
from langchain.vectorstores import Pinecone

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("YOUR_OPENAI_API_KEY") #you need to use your own ChatGPT API Key

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
# simple test
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]
res = chat(messages)
print(res.content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. It proposes that the fundamental building blocks of the universe are not particles but tiny, vibrating strings. These strings can have different modes of vibration, and the way they vibrate determines their properties, such as mass and charge.

String theory suggests that there are multiple dimensions beyond the four we experience in our everyday lives. It also predicts the existence of additional particles beyond those observed in the Standard Model of particle physics.

One of the key ideas in string theory is the concept of supersymmetry, which posits a relationship between particles with different spin values. Supersymmetry could potentially resolve some of the outstanding problems in particle physics, such as the hierarchy problem and the nature of dark matter.

However, string theory is still a work in progress and has not yet been experimentally confirmed. Res

## Create database of shipping status

In [5]:
import pandas as pd
shipping_data = [
    {
        "order_id": str(i),
        "customer_name": name,
        "status": status,
        "order_date": order_date,
        "shipping_date": shipping_date,
        "delivery_date": delivery_date,
        "shipping_address": address,
        "carrier": carrier,
        "tracking_number": tracking_number,
        "item_description": item,
        "quantity": quantity,
        "shipping_cost": cost,
        "shipper_name": shipper_name,
        "shipper_phone_number": shipper_phone,
        "current_location": address if status == "Shipped" else current_location
    }
    for i, (name, status, order_date, shipping_date, delivery_date, address, carrier, tracking_number, item, quantity, cost, shipper_name, shipper_phone, current_location) in enumerate([
        ("Alice Johnson", "Shipped", "2023-10-25", "2023-10-27", "2023-11-05", "123 Maple St, Chicago", "FedEx", "1Z9999", "Laptop", 1, 15.99, "John Doe", "+1-800-463-3339", "123 Maple St, Chicago"),
        ("Bob Smith", "Processing", "2023-10-28", "2023-10-29", "2023-11-07", "456 Oak St, New York", "UPS", "1Z9988", "Smartphone", 1, 10.50, "Sarah Lee", "+1-800-742-5877", "New York"),
        ("Charlie Brown", "Delivered", "2023-10-20", "2023-10-22", "2023-10-31", "789 Pine St, Los Angeles", "USPS", "9400", "Tablet", 1, 12.00, "Delivered", "+1-800-275-8777", "Los Angeles"),
        ("David Lee", "Shipped", "2023-10-26", "2023-10-29", "2023-11-06", "321 Cedar St, Houston", "DHL", "JD0001", "Headphones", 2, 8.75, "Michael Brown", "+1-800-225-5345", "321 Cedar St, Houston"),
        ("Eve Thompson", "Cancelled", "2023-10-29", "2023-10-30", "2023-10-30", "654 Elm St, Miami", "UPS", "1Z9977", "Smartwatch", 1, 0.00, "Cancelled", "+1-800-742-5877", "Miami"),
        ("Frank White", "Processing", "2023-10-30", "2023-10-31", "2023-11-08", "987 Birch St, Phoenix", "UPS", "1Z9966", "Camera", 1, 9.25, "Emily Smith", "+1-800-742-5877", "Phoenix"),
        ("Grace Miller", "Delivered", "2023-10-19", "2023-10-21", "2023-10-30", "111 Spruce St, San Francisco", "FedEx", "1Z8888", "Bluetooth Speaker", 1, 7.80, "Delivered", "+1-800-463-3339", "San Francisco"),
        ("Hannah Adams", "Shipped", "2023-10-27", "2023-10-30", "2023-11-09", "222 Walnut St, Seattle", "DHL", "JD0002", "E-reader", 1, 6.90, "Carlos Green", "+1-800-225-5345", "222 Walnut St, Seattle"),
        ("Isaac Clark", "Pending", "2023-10-31", "2023-11-01", "2023-11-10", "333 Chestnut St, Denver", "USPS", "9401", "Wireless Mouse", 3, 5.50, "Pending", "+1-800-275-8777", "Denver"),
    ])
]

# Convert to DataFrame for easy data manipulation
shipping_df = pd.DataFrame(shipping_data)

#shipping_df.to_csv("shipping_status.csv", index=False)



We begin by initializing our connection to Pinecone, this requires a [free API key](https://app.pinecone.io).

In [6]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="YOUR_API_KEY") #use your own Pinecone API Key
spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

Then we initialize the index. We will be using OpenAI's `text-embedding-ada-002` model for creating the embeddings, so we set the `dimension` to `1536`.

In [7]:
index_name = 'llama-2-rag'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 9}},
 'total_vector_count': 9}

embed and index all our dataset

In [8]:
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")


batch_size = 100

for i in tqdm(range(0, len(shipping_df), batch_size)):
    i_end = min(len(shipping_df), i + batch_size)
    batch = shipping_df.iloc[i:i_end]

    # Generate unique IDs for each record
    ids = [f"order-{x['order_id']}" for _, x in batch.iterrows()]

    # Extract detailed text to embed, including all key order information
    texts = [
        f"Order ID: {x['order_id']}, Customer: {x['customer_name']}, Status: {x['status']}, Shipper Name: {x['shipper_name']}, Shipper Phone Number: {x['shipper_phone_number']}, "
        f"Order Date: {x['order_date']}, Shipping Date: {x['shipping_date']}, Delivery Date: {x['delivery_date']}, "
        f"Shipping Address: {x['shipping_address']}, Carrier: {x['carrier']}, Tracking Number: {x['tracking_number']}, "
        f"Item: {x['item_description']}, Quantity: {x['quantity']}, Shipping Cost: ${x['shipping_cost']}, Current Location: {x['current_location']}"
        for _, x in batch.iterrows()
    ]

    # Generate embeddings for each record
    embeds = embed_model.embed_documents(texts)

    # Prepare metadata, ensuring each field is included
    metadata = [
        {
            'order_id': x['order_id'],
            'customer_name': x['customer_name'],
            'status': x['status'],
            'order_date': x['order_date'],
            'shipping_date': x['shipping_date'],
            'delivery_date': x['delivery_date'],
            'shipping_address': x['shipping_address'],
            'carrier': x['carrier'],
            'tracking_number': x['tracking_number'],
            'item_description': x['item_description'],
            'quantity': x['quantity'],
            'shipping_cost': x['shipping_cost'],
            'shipper_name': x['shipper_name'],
            'shipper_phone_number': x['shipper_phone_number'],
            'current_location': x['current_location'],
            'text': texts[j]  # Add the text used for embedding to metadata for reference
        }
        for j, x in batch.iterrows()
    ]

    # Upsert to Pinecone, with metadata containing detailed fields
    index.upsert(vectors=zip(ids, embeds, metadata))



/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
index.describe_index_stats

<bound method Index.describe_index_stats of <pinecone.data.index.Index object at 0x7fa0a93f86a0>>

In [14]:
from langchain.vectorstores import Pinecone


# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_key='text'
)
def augment_prompt(query: str):
  results = vectorstore.similarity_search(query, k=1)
  source_knowledge = "\n".join([x.page_content for x in results])
  augmented_prompt = f"""
  You are an AI assistant, you role is to support customer about their order status, provide exact and concise information about their order.
  We provided database about features of order status such as customer name, order day, shipper day, quantity, carrier.
  Ask customer about their order id and what information they want to know.
  You need to answer only the information about related to order statuses in database, if customer query unrelevant information, ask them to input another query.
  Be sure that you don't answer unrelevanted information.
  The following is the shipping status database:
  {shipping_df}
  If customer think that there's any somthing wrong or problem with their order, ask them to contact customer service at hamy95637@gmail.com
  Contexts:
  {source_knowledge}

  * customer_name: the name of the person receive the order
  * status: the current order status, is the order being shipped, delivered, pending or cancellled?
  * order_date: the date that the order is ordered
  * delivery_date: the date that the order is or will be expected to be shipped or done.
  * shipping_date: the date that the seller transfer the order to shipper, and the order move.
  * shipping_adress: the order is shipped to this adress
  * carrier: shipping unit which is responsible for delivery
  * tracking_number: the unique number that the admin use for tracking more details of the order if there is any problem.
  * item_description: one of the information on the contents of the package. It is the name of item the customer ordered.
  * quantity: one of the information on the contents of the package. It express the number of item in the package ordered.
  * shipping_cost: the cost of the order the customer must pay.
  * shipper_name: the name of the person deliver your order
  * shipper_phone_number: the phone number of your shipper, you can contact with the shipper by this.
  * current_location: the location of the order in realtime, if the order was shipped, this location will be the shipping address.
  Query: {query}"""
  return augmented_prompt


/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [15]:
messages = [
    SystemMessage(content="You are an assistant that provides short and concise shipping status updates based on order information."),
    HumanMessage(content="Hi AI, can you tell me about my shipping status?"),
    AIMessage(content="Sure, can you tell me your order id?"),
    HumanMessage(content="My order id is 0."),
    AIMessage(content="what do you want to know about your order"),
    HumanMessage(content="I want to know about shipping cost"),
    AIMessage(content="Your shipping cost of order id 0 is 15.99"),
    HumanMessage(content="ok, i want to know about another order"),
    AIMessage(content="Sure, can you tell me your order id?")
]
while True:
    # Get the user question
    query = input()

    # Check if the user wants to exit
    if query.lower() == 'exit':
        print("Exiting the program. Goodbye!")
        break

    # Generate prompt based on the question
    prompt = HumanMessage(content=augment_prompt(query))

    # Add to messages
    messages.append(prompt)

    # Send to OpenAI model and get response
    res = chat(messages)
    print("Response:", res.content)

hello
Response: I'm sorry, I didn't understand your query. Please provide your order id and the information you would like to know about your order.
my order id and i want to know when my order will be come
Response: Please provide me with your order ID so I can check the expected delivery date for your order.
my order id is 1
Response: Your order with id 1 is currently in the processing status. It was ordered on 2023-10-28, shipped on 2023-10-29, and is expected to be delivered by 2023-11-07. The carrier is UPS and the item is a Smartphone. If you need further assistance, feel free to ask.
how much
Response: I'm sorry, but could you please provide a specific question related to your order status?
how much money 
Response: Please clarify your query so I can provide you with the relevant information regarding your order.
how much money the packgae cost
Response: The package for order ID 7 costs $6.90.
but my order is 1
Response: The shipping cost for your order 1 is $10.50.
are you sure